In [1]:
import numpy as np
np.random.seed(42) # for repeatability 
theta_true = (25, 0.5)
xdata = 100 * np.random.random(20)
ydata = theta_true[0] + theta_true[1] * xdata 
ydata = np.random.normal(ydata, 10) # add error

### Frequentist Solution

In [2]:
X = np.vstack([np.ones_like(xdata), xdata]).T

In [3]:
theta_hat = np.linalg.solve(np.dot(X.T, X), np.dot(X.T, ydata))

In [4]:
y_hat = np.dot(X, theta_hat)

In [5]:
sigma_hat = np.std(ydata - y_hat)

In [6]:
Sigma = sigma_hat ** 2 * np.linalg.inv(np.dot(X.T, X))

In [7]:
import statsmodels.api as sm

In [8]:
X = sm.add_constant(xdata)
result = sm.OLS(ydata, X).fit()
sigma_hat = result.params
Sigma = result.cov_params()
print(result.summary2())

                 Results: Ordinary least squares
Model:              OLS              Adj. R-squared:     0.683   
Dependent Variable: y                AIC:                147.7737
Date:               2019-04-07 23:29 BIC:                149.7651
No. Observations:   20               Log-Likelihood:     -71.887 
Df Model:           1                F-statistic:        41.97   
Df Residuals:       18               Prob (F-statistic): 4.30e-06
R-squared:          0.700            Scale:              86.157  
-------------------------------------------------------------------
            Coef.    Std.Err.     t      P>|t|     [0.025    0.975]
-------------------------------------------------------------------
const      24.6361     3.7871   6.5053   0.0000   16.6797   32.5924
x1          0.4483     0.0692   6.4782   0.0000    0.3029    0.5937
-----------------------------------------------------------------
Omnibus:              1.996        Durbin-Watson:           2.758
Prob(Omnibus):   

### Bayesian MCMC

In [9]:
import emcee # version 2.0 

def log_prior(theta):
    alpha, beta, sigma = theta 
    if sigma < 0:
        return -np.inf # log(0) 
    else:
        return (-1.5 * np.log(1 + beta**2) - np.log(sigma))
    
def log_like(theta, x, y):
    alpha, beta, sigma = theta
    y_model = alpha + beta * x
    return -0.5 * np.sum(np.log(2*np.pi*sigma**2) + (y-y_model)**2 / sigma**2) 

def log_posterior(theta, x, y):
    return log_prior(theta) + log_like(theta,x,y)

In [10]:
ndim = 3 # number of parameters in the model 
nwalkers = 50 # number of MCMC walkers
nburn = 1000 # "burn-in" to stabilize chains 
nsteps = 2000 # number of MCMC steps to take 
starting_guesses = np.random.rand(nwalkers, ndim)

In [11]:
sampler = emcee.EnsembleSampler(nwalkers, ndim,
                                log_posterior,
                                args=[xdata,ydata])
sampler.run_mcmc(starting_guesses, nsteps)
# chain is of shape (nwalkers, nsteps, ndim): # discard burn-in points and reshape:
trace = sampler.chain[:, nburn:, :]
trace = trace.reshape(-1, ndim).T